[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Datacompintensive/WignerCamp2025/blob/master/ALT/presentation_calculation.ipynb)

# Introduction to ALT – A Simple Numerical Demonstration

This notebook introduces the **Adaptive Law-based Transformation (ALT)** with a minimal numerical example. ALT is a lightweight, interpretable, and effective method for **time series classification**. It extracts characteristic patterns—called *shapelets*—from the training data and uses them to transform new data into a feature space where classes are more easily separable.

We demonstrate each step using a synthetic example with only a few elements, allowing full transparency into how ALT works under the hood.

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib as mpl

from functions import *

In [2]:
%load_ext autoreload
%autoreload 2

## Step 1: Define a minimal training and test set
We generate synthetic time series using simple **linear recursion rules**:
- Class `a` is generated using the Fibonacci rule: $x_n = x_{n-1} + x_{n-2}$. ($w=[1, 1]$).
- Class `b` uses a different rule: $x_n = 2x_{n-2} + x_{n-1}$. ($w=[2, 1]$).
- The test sample also follows the Fibonacci rule but starts from different initial values.

In [3]:
a = generate_recursive_array([1, 1], [1, 1], 5)
print(f"Train instance for class 'a': {a}")
b = generate_recursive_array([2, 1], [2, 1], 5)
print(f"Train instance for class 'b': {b}")
x = generate_recursive_array([1, 1], [2, 3], 5)
print(f"Test instance 'x': {x} belonging to an unknown class.")

Train instance for class 'a': tensor([1., 1., 2., 3., 5.])
Train instance for class 'b': tensor([ 2.,  1.,  5.,  7., 17.])
Test instance 'x': tensor([ 2.,  3.,  5.,  8., 13.]) belonging to an unknown class.


## Step 2: Extract Laws from Training Data
We extract **shapelet vectors** (or 'laws') from each training instance. Each law is a direction in space that characterizes how values change locally.

We use parameters $R=3$, $L=2$, $K=1$, meaning:
- $R=3$: the length of subsequences considered.
- $L=2$: the size of the matrix constructed from those sequences.
- $K=1$: the shift between consecutive windows.

The shapelet vectors are obtained by computing the eigenvector of a symmetric matrix (constructed via time-delay embedding) corresponding to the **smallest eigenvalue**.

In [4]:
laws_a = extract_symmetric_laws(a)
laws_b = extract_symmetric_laws(b)
print(f"Laws for class 'a':\n{laws_a}")
print(f"Laws for class 'b':\n{laws_b}")

Laws for class 'a':
tensor([[-0.8507, -0.8507, -0.8507],
        [ 0.5257,  0.5257,  0.5257]])
Laws for class 'b':
tensor([[-0.9571, -0.8702, -0.9085],
        [ 0.2898,  0.4927,  0.4179]])


## Step 3: Embed the Test Instance
We now embed the test instance into 2D vectors (pairs of values) that will be multiplied by the laws. This step constructs an input matrix $E$.

In [5]:
embedded = embed_as_pairs(x)
print(f"The embedded test instance: \n{embedded}")

The embedded test instance: 
tensor([[ 2.,  3.],
        [ 3.,  5.],
        [ 5.,  8.],
        [ 8., 13.]])


## Step 4: Apply Laws (Matrix Multiplication)
We apply the transformation by multiplying the embedded matrix with the laws of each class. The goal is that the *correct laws* will make the result close to zero vectors, while *incorrect laws* won’t.

In [6]:
F_a = embedded @ laws_a
F_b = embedded @ laws_b
print(f"Multiplying with laws from class 'a' gives:\n{F_a}")
print(f"Multiplying with laws from class 'b' gives:\n{F_b}")

Multiplying with laws from class 'a' gives:
tensor([[-0.1241, -0.1241, -0.1241],
        [ 0.0767,  0.0767,  0.0767],
        [-0.0474, -0.0474, -0.0474],
        [ 0.0293,  0.0293,  0.0293]])
Multiplying with laws from class 'b' gives:
tensor([[-1.0448, -0.2623, -0.5635],
        [-1.4224, -0.1471, -0.6363],
        [-2.4672, -0.4094, -1.1997],
        [-3.8895, -0.5565, -1.8360]])


## Step 5: Feature Extraction from Transformed Matrix
From the result of the transformation, we compute a simple **statistical feature**: the average of the squared values (mean of $F^2$). Lower values indicate better alignment with the laws of a class.

In [7]:
res_a = torch.mean(F_a**2)
res_b = torch.mean(F_b**2)
print(f"For class 'a': mean_all: {res_a}")
print(f"For class 'b': mean_all: {res_b}")
print(f"\nBased on the method the unknown instance belongs to class: '{'a' if res_a < res_b else 'b'}'.")

For class 'a': mean_all: 0.00609796354547143
For class 'b': mean_all: 2.5358715057373047

Based on the method the unknown instance belongs to class: 'a'.


## Summary
In this minimal example, ALT successfully classified the test sequence. The method:
- Extracted characteristic patterns (laws) from the training classes.
- Transformed the new sequence into a comparison-friendly space.
- Used simple statistics to determine which class's laws best describe the new data.

This illustrates how **ALT builds an interpretable and efficient classification pipeline**, leveraging linear algebra and time-delay embeddings.

For more advanced applications, ALT supports multiple channels, longer sequences, and sophisticated feature extraction techniques like percentiles and higher moments.